#Day 63,64 (2025-12-10,11)
## Extract

In [0]:
# NOT requuired in databricks notebook since spark session is already created and available
from pyspark.sql import SparkSession
spark1 = SparkSession.builder.appName("Car Demo").getOrCreate()
print(id(spark) , " - ID of spark created and given to us by Databricks")
print(id(spark1), " - ID of spark created by us; its not created but get from existing instance by SparkSession.builder.getOrCreate() method")
print("Both IDs will be same")

## Programatically create schema, volume and folder

## Directory structure / Hierarchy 
### catalog/workspace -> Schema/database -> volume -> directory/folder -> data/files

In [0]:
%sql
-- Catalog / Workspace
create catalog if not exists catalog1_dropme;

-- Schema / Database
-- create schema if not exists catalog1_dropme.schema1;
-- OR
create database if not exists catalog1_dropme.schema1;

--volume
create volume if not exists catalog1_dropme.schema1.volume1;

-- folders/directory will be created using file system %fs  / dbutils

In [0]:
dbutils.fs.mkdirs("/Volumes/catalog1_dropme/schema1/volume1/source")
dbutils.fs.put("/Volumes/catalog1_dropme/schema1/volume1/source/test.txt","something")

In [0]:
%sql
create schema if not exists wd36schema;
create volume if not exists wd36schema.ingestion_volume;

In [0]:
dbutils.fs.mkdirs("/Volumes/workspace/wd36schema/ingestion_volume/source")
dbutils.fs.put("/Volumes/workspace/wd36schema/ingestion_volume/source/test.txt","something")

## Read data from csv files

### CSV - using spark.read.csv -> "Dataframe"
(path: PathOrPaths)

<br><br>
(path: PathOrPaths, schema: Optional[Union[StructType, str]]=None, sep: Optional[str]=None, encoding: Optional[str]=None, quote: Optional[str]=None, escape: Optional[str]=None, comment: Optional[str]=None, header: Optional[Union[bool, str]]=None, inferSchema: Optional[Union[bool, str]]=None, ignoreLeadingWhiteSpace: Optional[Union[bool, str]]=None, ignoreTrailingWhiteSpace: Optional[Union[bool, str]]=None, nullValue: Optional[str]=None, nanValue: Optional[str]=None, positiveInf: Optional[str]=None, negativeInf: Optional[str]=None, dateFormat: Optional[str]=None, timestampFormat: Optional[str]=None, maxColumns: Optional[Union[int, str]]=None, maxCharsPerColumn: Optional[Union[int, str]]=None, maxMalformedLogPerPartition: Optional[Union[int, str]]=None, mode: Optional[str]=None, columnNameOfCorruptRecord: Optional[str]=None, multiLine: Optional[Union[bool, str]]=None, charToEscapeQuoteEscaping: Optional[str]=None, samplingRatio: Optional[Union[float, str]]=None, enforceSchema: Optional[Union[bool, str]]=None, emptyValue: Optional[str]=None, locale: Optional[str]=None, lineSep: Optional[str]=None, pathGlobFilter: Optional[Union[bool, str]]=None, recursiveFileLookup: Optional[Union[bool, str]]=None, modifiedBefore: Optional[Union[bool, str]]=None, modifiedAfter: Optional[Union[bool, str]]=None, unescapedQuoteHandling: Optional[str]=None) -> "DataFrame"

### CSV - using spark.read.csv -> "Dataframe"
/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1 -> comma(',') seperted with no header
/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header -> comma(',') seperted with header
/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header_oth_del -> other delimiter ('~') with header

<br>
#### 1. path: PathOrPaths; No other options
#### By default, 
##### separator / delimiter is ','
##### header as _c0, _c1..._cn; n-> no of columns
##### all columns aas string type 
<br>


In [0]:
%python
# data with comma seperated and NO column header
csv_df1 =spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1")
print(csv_df1.printSchema())
display(csv_df1)

In [0]:
# data with comma seperated and column header
csv_df1a =spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header")
print(csv_df1a.printSchema())
display(csv_df1a)

### other delimiter
#### If seperator not provided, everting will be consider as single column (if no comma present in the data)

In [0]:
%python
csv_df2=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header_oth_del")
print(csv_df2.printSchema())
display(csv_df2)

##### with seperator option

In [0]:
csv_df2a=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header_oth_del",sep='~')
print(csv_df2a.printSchema())
display(csv_df2a)

### Header concepts
#### Define our own column names using toDF fnction by providing teh header columns
##### instead of default _c0,_c1... define our own column names
#### Use column names in the data; enabling header=True by considering first row as header columns


In [0]:
# data with comma seperated and NO column header
csv_df1 =spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1").toDF("custid","fname","lname","age","profession")
print(csv_df1.printSchema())
csv_df1.show(2)
#display(csv_df1)

In [0]:
# data with comma seperated and column header
csv_df1a =spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header", header=True )
print(csv_df1a.printSchema())
csv_df1a.show(2)
#display(csv_df1a)

## Infer Schema
### **Performance Consideration** need more causious to use this function
### It scans entire data if sampling not given and **imediately evaluating and executing** 
#### NOT good for large data 
#### NOT good to use in predefined schema datasets

In [0]:
csv_df1 =spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1").toDF("custid","fname","lname","age","profession")
print(csv_df1.printSchema())
csv_df1.where("custid in (4004979,4004981)").show()
csv_df1 =spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1",inferSchema=True).toDF("custid","fname","lname","age","profession")
print(csv_df1.printSchema())
csv_df1.where("custid in (4004979,4004981)").show()
print("******************************************")
csv_df1 =spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_2").toDF("custid","fname","lname","age","profession")
print(csv_df1.printSchema())
csv_df1.where("custid in (4004979,4004981)").show()
csv_df1 =spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_2",inferSchema=True).toDF("custid","fname","lname","age","profession")
print(csv_df1.printSchema())
csv_df1.where("custid in (4004979,4004981)").show()
#display(csv_df1)

## Delimiter / Seperator options

In [0]:
csv_df2=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header_oth_del").toDF("my_col_name")
print(csv_df2.printSchema())
csv_df2.show(2)

csv_df2.createOrReplaceTempView("csv_df2")
spark.sql("desc csv_df2").show()

spark.sql("select my_col_name from csv_df2").show()

spark.sql("select split(my_col_name,'~') as splitted from csv_df2").show()

In [0]:
csv_df2=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header_oth_del", header=True)
print(csv_df2.printSchema())
csv_df2.show(2)

csv_df2.createOrReplaceTempView("csv_df2")
spark.sql("desc csv_df2").show(truncate=False)

spark.sql("select * from csv_df2").show(truncate=False)

spark.sql("select `custid~fname~lname~age~profession` from csv_df2").show(truncate=False)

spark.sql("select split(`custid~fname~lname~age~profession`,'~') as splitted from csv_df2").show(truncate=False)

spark.sql("select split(`custid~fname~lname~age~profession`,'~') as splitted from csv_df2").createOrReplaceTempView("csv_df2a")
spark.sql("select splitted[0],splitted[1],splitted[2],splitted[3],splitted[4] from csv_df2a").show(2)

In [0]:
csv_df2a=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header_oth_del", header=True, sep='~')
print(csv_df2a.printSchema())
csv_df2a.show(2)

### Generic read option other than usal formats(csv,json,xml,..) like bigquery, redshift,athena, synapse...tmp folder, access control..

In [0]:
csv_df3=spark.read.options(header=True, inferSchema=True, sep='~').format('csv').load("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header_oth_del")
print(csv_df3.printSchema())
csv_df3.show(2)

### Read from multiple files
#### Read multiple files from a path (using regular expression/pathGlobFilter)
#### Read multiple files from multiple paths (using list of paths)
#### Read multiple files from path(s) and sub directories (using recurviseFileLookup)

In [0]:

csv_df1 = spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1", header=True, inferSchema=True, sep='~')
print("1", csv_df1.count())
csv_df2 = spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_2", header=True, inferSchema=True, sep='~')
print("2", csv_df2.count())
csv_df3 = spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header", header=True, inferSchema=True, sep='~')
print("3", csv_df3.count())
csv_df4 = spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_header_oth_del", header=True, inferSchema=True, sep='~')
print("4", csv_df4.count())

csv_df5 = spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs*", header=True, inferSchema=True, sep='~')
print("5", csv_df5.count())

csv_df6 = spark.read.csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1","/Volumes/workspace/wd36schema/ingestion_volume/source/custs_2"], header=True, inferSchema=True, sep='~')
print("6", csv_df6.count())

csv_df7 = spark.read.csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/NY/","/Volumes/workspace/wd36schema/ingestion_volume/source/TX/"], header=True, inferSchema=True, sep=',', pathGlobFilter="custs_header_*", recursiveFileLookup=True)
print("7", csv_df7.count())  
csv_df7.show(2)


## Custom Schema - Define the schema structure
### Using String Structure
### Using StructType

#### Using String Structure

In [0]:
csv_df1 = spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1", header=True, inferSchema=True).toDF("id","fname","lname","age","profession")
print(csv_df1.printSchema())
csv_df1.show(2)


str_struct = "id integer,fname string,lname string,age integer,profession string"

csv_df2 = spark.read.schema(str_struct).csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1", header=True)
print(csv_df2.printSchema())
csv_df2.show(2)

#### Using  StructType and DataType class
##### Steps to define the cutom schema usinfg StructTyre
1. Import StructType and DataType libraries/classes<br>
2. Intanstiate StructType object<br>
3. Create StructFiled object for each column which contains column name, data type and nullable True/False<br>
4. Pass as list argument to teh StructType object
e.g **from pyspark.sql.type import StructType,StructFiled,IngeterType,StringType...** <br>
define_structure = StructType([StructField('column_name1, DataType(), False, StructField('column_name2, DataType(), True)),...])


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
custom_schema = StructType([
    StructField("id", IntegerType(), False), 
    StructField("fname", StringType(), True), 
    StructField("lname", StringType(), True), 
    StructField("age", IntegerType(), True), 
    StructField("profession", StringType(), True)
])

csv_df7 = spark.read.schema(custom_schema).csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custs_1", header=True,enforceSchema=True)
print(csv_df7.printSchema())
csv_df7.show(2)